In [16]:
import os 
import re
import glob
import pandas as pd
import shutil
import unidecode
import numpy as np

In [17]:
def get_names_and_dates(file_names):
    #quebrando em sequencias de string
    tolkens = re.findall('([A-Z]+)(?=[\s-]+)', file_names)
    bad_tolkens = ["S0", "S1", "S2","S3", "S4", "S5","I0", "I1", "I2","I3", "I4", "I5", "OT", "SR", "CR", "PX"]
    #limpando os tolkens
    
    tolkens = list(filter((lambda piece: piece not in bad_tolkens),tolkens))
    
    #juntando os nomes
    full_name = ""
    for index, name in enumerate(tolkens):
        if index == 0:
            full_name += name
        else:
            full_name += " "+name
    
    #pegando as datas
    dates = re.findall("\d\d-\d\d-\d{4}", file_names)
    
    return (full_name, dates)
    

In [18]:
def get_name(file_name):

    tolkens = re.findall('([A-Z]+)(?=[\s-]+)', file_name)
    
    bad_tolkens = ["S0", "S1", "S2","S3", "S4", "S5","I0", "I1", "I2","I3", "I4", "I5", "OT", "SR", "CR", "PX"]

    tolkens = list(filter((lambda piece: piece not in bad_tolkens),tolkens))
    
    full_name = ""
    for index, name in enumerate(tolkens):
        if index == 0:
            full_name += name
        else:
            full_name += " " + name
        
    return (full_name)

In [19]:
def indentify_files(file_name):
    tolkens = re.findall('([A-Z]+)(?=[\s-]+)', file_name)
    
    file_type = "Unknown"
    
    if ("CR" in tolkens) or ("SR" in tolkens):
        if re.findall("S0", file_name):
            file_type = "dxa"
        else:
            file_type = "body-xray"
    elif "OT" in tolkens:
        file_type = "dxa"
    elif "PX" in tolkens:
        file_type ="pan"
    elif re.findall('.seg.nrrd', file_name):
        file_type="segmentation"
        
    return file_type
    

In [20]:
def data_diff (date1,date2):
    patern = "[\d]+"
    
    dates1= re.findall(patern, date1)
    dates2= re.findall(patern, date2)
    
    days= int(dates2[0]) - int(dates1[0])
    months= int(dates2[1])- int(dates1[1])
    years= int(dates2[2])- int(dates1[2])
    
    diff_years = round(float(days + months*30)/365.0 + float(years), 2)
    diff_months = round(float(days/30 + months + years*12),2)
    diff_days = float(days + months*30 +years*365.0)
    
    return diff_years, diff_months, diff_days

# Listando Nomes, dob, data_pan, pan_path dos pacientes

In [21]:
main_folder = r'/home/davi/IC DAVI/Image_for_segmentation'

#listando todos os pacientes da pasta
patients = os.listdir(main_folder)

pan_files=[]
pan_info=[]

for patient in patients:
    patient_folder = os.path.join(main_folder, patient)
    print(patient_folder)
    
    patient_files=os.listdir(patient_folder)
    
    #selecionando as panoramicas .BMP
    
    for file in patient_files:
        if file.endswith(".BMP"):
            #identificando se o arquivo é pan
            if indentify_files(file)== 'pan':
                try:
                    info=get_names_and_dates(file)
                    patient_name= info[0]
                    dob=info[1][0]
                    pan_date=info[1][1]
                    age_at_pan=data_diff(dob, pan_date)[2]
                    pan_path = os.path.join(patient_folder, file)
                
                    pan_info.append([patient_name, dob, pan_date, age_at_pan, pan_path])
                
                except IndexError as error:
                    print("vamos ver")
                    print(error)
                

    pan_df = pd.DataFrame(pan_info, columns=['Patient', 'Date of birth', 'PAN date', 'Age at PAN', 'PAN path'])


/home/davi/IC DAVI/Image_for_segmentation/APARECIDA DE SOUSA NETO
/home/davi/IC DAVI/Image_for_segmentation/ELAINE CRISTINA DA SILVA
/home/davi/IC DAVI/Image_for_segmentation/LUCIA TAVARES ESTEVES
/home/davi/IC DAVI/Image_for_segmentation/MARCILIO RODRIGUES
/home/davi/IC DAVI/Image_for_segmentation/OSVALDO DONIZETI DA SILVA
/home/davi/IC DAVI/Image_for_segmentation/ANA HELENA MACHADO
/home/davi/IC DAVI/Image_for_segmentation/IRMAO JOSE F DE MELO
/home/davi/IC DAVI/Image_for_segmentation/TERESA CRISTINA LOPES GUIMARAES
/home/davi/IC DAVI/Image_for_segmentation/SAUL DE CASTRO SILVA
/home/davi/IC DAVI/Image_for_segmentation/BIGARAM LAERCIO
/home/davi/IC DAVI/Image_for_segmentation/JOSE DA COSTA
/home/davi/IC DAVI/Image_for_segmentation/MILTON XAVIER BONFIM JUNIOR
/home/davi/IC DAVI/Image_for_segmentation/ANTONIO CARLOS SOUZA SILVA
/home/davi/IC DAVI/Image_for_segmentation/CLEUSA DOS SANTOS SIMAO
/home/davi/IC DAVI/Image_for_segmentation/BRITO LAZARO TEIXEIRA DE
/home/davi/IC DAVI/Image_fo

/home/davi/IC DAVI/Image_for_segmentation/VALDECI CARDOSO RIBEIRO
/home/davi/IC DAVI/Image_for_segmentation/ANTONIO ROBERTO WALDERRAMA
/home/davi/IC DAVI/Image_for_segmentation/CASAQUIA- LAERTE
/home/davi/IC DAVI/Image_for_segmentation/JOAO VICENTE DE OLIVEIRA
/home/davi/IC DAVI/Image_for_segmentation/JOSE BATISTA DE MOURA
/home/davi/IC DAVI/Image_for_segmentation/ILIZIO BRUGNOLLI
/home/davi/IC DAVI/Image_for_segmentation/MADALENA DA SILVA
/home/davi/IC DAVI/Image_for_segmentation/JURACY ALVES DE LIMA SOUSA
/home/davi/IC DAVI/Image_for_segmentation/DIAS NEUCY DE OLIVEIRA
/home/davi/IC DAVI/Image_for_segmentation/JOÃO TEIXEIRA BARBOSA
/home/davi/IC DAVI/Image_for_segmentation/APARECIDO GOMES DA SILVA(so PAN)
/home/davi/IC DAVI/Image_for_segmentation/REGIS MARIANO NASCIMENTO
/home/davi/IC DAVI/Image_for_segmentation/GRIFI LUIS CARLOS
/home/davi/IC DAVI/Image_for_segmentation/PAULO SILAS MENDES
/home/davi/IC DAVI/Image_for_segmentation/CLEIDE PEREIRA
/home/davi/IC DAVI/Image_for_segmentat

/home/davi/IC DAVI/Image_for_segmentation/ANGELA MARIA PIANCO DE PAULA
/home/davi/IC DAVI/Image_for_segmentation/CLAUDIA REIS DA SILVA
/home/davi/IC DAVI/Image_for_segmentation/ANTONIO RAYMUNDO DA SILVA (Só PAN)
/home/davi/IC DAVI/Image_for_segmentation/ROSIMEIRE PINHEIRO DE FREITAS
/home/davi/IC DAVI/Image_for_segmentation/JOSE MIGUEL CIRILO
/home/davi/IC DAVI/Image_for_segmentation/JOSE CALEGARO
/home/davi/IC DAVI/Image_for_segmentation/ANTONIO PRAXEDES DA SILVA
/home/davi/IC DAVI/Image_for_segmentation/LUCILA BALDINI PUGAS
/home/davi/IC DAVI/Image_for_segmentation/SEBASTIAO MARCELINO COSTA
/home/davi/IC DAVI/Image_for_segmentation/ELENI COSTA
/home/davi/IC DAVI/Image_for_segmentation/VALERIA CRISTINA ALVES PAIXAO
/home/davi/IC DAVI/Image_for_segmentation/AURO MACHADO
/home/davi/IC DAVI/Image_for_segmentation/ADOVIDIO FLORENCIO SILVA
/home/davi/IC DAVI/Image_for_segmentation/NASCIMENTA MAGDALENA FERREIRA
/home/davi/IC DAVI/Image_for_segmentation/CASAQUIA LAERTE
/home/davi/IC DAVI/Ima

/home/davi/IC DAVI/Image_for_segmentation/LUZIA CALTRAN
/home/davi/IC DAVI/Image_for_segmentation/VALDIR JOSE FERREIRA
/home/davi/IC DAVI/Image_for_segmentation/JOSE NOVEMBRE
/home/davi/IC DAVI/Image_for_segmentation/LEONICE ALVES FERREIRA SIQUEIRA
/home/davi/IC DAVI/Image_for_segmentation/BOARO SILVIO CELSO
/home/davi/IC DAVI/Image_for_segmentation/LUCILENE FERNANDES DE GOUVEA
/home/davi/IC DAVI/Image_for_segmentation/MARIA SOCORRO DOS SANTOS GOMES
/home/davi/IC DAVI/Image_for_segmentation/GONCALVES KLEBER WENDEL
/home/davi/IC DAVI/Image_for_segmentation/FILHO JOAO BRANCO FARIA
/home/davi/IC DAVI/Image_for_segmentation/ANTONIO GUILHERMINO
/home/davi/IC DAVI/Image_for_segmentation/SERGIO SANTOS
/home/davi/IC DAVI/Image_for_segmentation/MARCOS ANTONIO DE OLIVEIRA CORREA
/home/davi/IC DAVI/Image_for_segmentation/OSMAR ALVES MACHADO
/home/davi/IC DAVI/Image_for_segmentation/RACHEL GONÇALVES DE OLIVEIRA
/home/davi/IC DAVI/Image_for_segmentation/CHICORIA- MARIA- SOCORRO DOS SA
/home/davi/IC

In [23]:
pan_df
pan_df.to_csv('Dados.csv', index=False)